In [1]:
import os
import time

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle

from costCalcuation.distributions.create_distribtion_helper import create_helper_for_distribution
from depth_first_search_solver import DepthFirstSearchSolver
from experiment_details import ExperimentDetails
from parse_input import parse_xml, parse_itc2007_curriculum_based, parse_itc2007_post_enrolment
from solution_search import SolutionSearch
from costCalcuation.distributions.create_distribtion_helper import create_helper_for_distribution




In [2]:
itc2007_track2_path = 'D:\\Desktop\\Datasets\\post'
itc2007_track3_path = 'D:\\Desktop\\Datasets\\curriculum'
itc2019_path = 'D:\\Desktop\\Datasets\\2019'

In [3]:
def get_all_files(path):
    return [os.path.join(dirpath, file) for dirpath, _, filenames in os.walk(path) for file in filenames]    

In [4]:
itc2007_track2_files = get_all_files(itc2007_track2_path)
itc2007_track3_files = get_all_files(itc2007_track3_path)
itc2019_files = get_all_files(itc2019_path)

# Get statistics

## Statistics for ITC2019

In [ ]:
problems_and_stats = [parse_xml(file) for file in itc2019_files]
stats = [problem[1] for problem in problems_and_stats]

df = pd.DataFrame(stats)

df.index += 1
df.reset_index(inplace=True,names='index')

os.makedirs('output/stats', exist_ok=True)
df.to_csv('output/stats/itc2019_stats.csv',index=False)


## Statistics for ITC2007 Track 2

In [ ]:
problems_and_stats = [parse_itc2007_post_enrolment(file) for file in itc2007_track2_files]
stats = [problem[1] for problem in problems_and_stats]

df = pd.DataFrame(stats)


#make index 1 based and name them as their file names ex. 1,2,...
df.index += 1
df.reset_index(inplace=True,names='name')

df.index += 1
df.reset_index(inplace=True,names='index')

os.makedirs('output/stats', exist_ok=True)
df.to_csv('output/stats/itc2007_track2_stats.csv',index=False)

## Statistics for ITC2007 Track 3

In [ ]:
problems_and_stats = [parse_itc2007_curriculum_based(file) for file in itc2007_track3_files]
stats = [problem[1] for problem in problems_and_stats]

df = pd.DataFrame(stats)


df.index += 1
df.reset_index(inplace=True,names='index')

os.makedirs('output/stats', exist_ok=True)
df.to_csv('output/stats/itc2007_track3_stats.csv',index=False)


# Construction using depth first search

In [5]:
def plot_row_over_operation(file,operation_history , title):
    fig,ax = plt.subplots()
    ax.set_title(title)
    
    y = [i['current_row'] for i in operation_history]
    
    ax.set_xlabel('Operation')
    ax.set_ylabel('Classes placed')
    
    ax.plot(y)
    plt.show()
    
    if file:
        fig.savefig(file + '.png')
    
    
    
def plot_row_over_time(file, operation_history , title):
    fig,ax = plt.subplots()
    ax.set_title(title)
    
    time = [i['time'] for i in operation_history]
    row = [i['current_row'] for i in operation_history]
    
    ax.set_xlabel('Time')
    ax.set_ylabel('Classes placed')
    ax.plot(time,row)

    plt.show()
    
    if file:
        fig.savefig(file + '.png')
    

In [ ]:
folder = 'output/itc2007_track2_depth_first_search_test_'+time.strftime("%Y%m%d-%H%M%S")
os.mkdir(folder)

for f in itc2007_track2_files:
    
    sub_folder = os.path.join(folder, f.split('\\')[-1].split('.')[0])
    os.mkdir(sub_folder)
    
    date = time.time()
    
    problem = parse_itc2007_post_enrolment(f)[0]
    search = SolutionSearch(problem)
    solver = DepthFirstSearchSolver(search)
    res = solver.solve(max_operations=10000)
    
    experiment_details = ExperimentDetails(
        name='depth_first_search with 10k max operations',
        instance_name=f.split('\\')[-1].split('.')[0],
        instance_file_name = f.split('\\')[-1].split('.')[0],
        dataset_name='itc2007_track2',
        date=date,
        results_data=res,
        experiment_data={
            "solver": 'depth_first_search',
            "max_operations": 10000
        }
    )
    
    pickle.dump(experiment_details, open(os.path.join(sub_folder, 'experiment_details.pkl'), 'wb'))
    
    if res['success']:
        np.save(os.path.join(sub_folder, 'solution gene.npy'), search.get_result_as_gene())
    
    plot_row_over_operation(os.path.join(sub_folder, 'row_over_operation_graph.png'), res['operation_history'], 'Depth first search max 10k operations for itc2007_track2 - ' + experiment_details.instance_name)
    
    plot_row_over_time(os.path.join(sub_folder, 'row_over_time_graph.png'),res['operation_history'], 'Depth first search max 10k operations for itc2007_track2 - ' + experiment_details.instance_name)
    

In [ ]:
folder = 'output/itc2007_track3_depth_first_search_test_'+time.strftime("%Y%m%d-%H%M%S")
os.mkdir(folder)

for f in itc2007_track3_files:
    
    
    
    date = time.time()
    
    problem = parse_itc2007_curriculum_based(f)[0]
    search = SolutionSearch(problem)
    solver = DepthFirstSearchSolver(search)
    res = solver.solve(max_operations=10000)
    
    sub_folder = os.path.join(folder, problem.name)
    os.mkdir(sub_folder)
    
    experiment_details = ExperimentDetails(
        name='depth_first_search with 10k max operations',
        instance_name=problem.name,
        instance_file_name = f.split('\\')[-1].split('.')[0],
        dataset_name='itc2007_track3',
        date=date,
        results_data=res,
        experiment_data={
            "solver": 'depth_first_search',
            "max_operations": 10000
        }
    )
    
    pickle.dump(experiment_details, open(os.path.join(sub_folder, 'experiment_details.pkl'), 'wb'))
    
    if res['success']:
        np.save(os.path.join(sub_folder, 'solution gene.npy'), search.get_result_as_gene())
    
    plot_row_over_operation(os.path.join(sub_folder, 'row_over_operation_graph.png'), res['operation_history'], 'Depth first search max 10k operations for itc2007_track3 - ' + experiment_details.instance_name)
    
    plot_row_over_time(os.path.join(sub_folder, 'row_over_time_graph.png'),res['operation_history'], 'Depth first search max 10k operations for itc2007_track3 - ' + experiment_details.instance_name)
    

In [6]:
folder = 'output/itc2019_depth_first_search_test_'+time.strftime("%Y%m%d-%H%M%S")
os.mkdir(folder)

for f in itc2019_files[5:6]:
#for f in ['D:\\Desktop\\Datasets\\test\\2019maxblock.xml']: 
    date = time.time()
    
    problem = parse_xml(f)[0]
    search = SolutionSearch(problem)
    solver = DepthFirstSearchSolver(search)

    try:
        res = solver.solve(max_operations=10000)
    except Exception as e:
        res = {'success': False, 'error': str(e)}
    
    sub_folder = os.path.join(folder, problem.name)
    os.mkdir(sub_folder)
    
    experiment_details = ExperimentDetails(
        name='depth_first_search with 10k max operations',
        instance_name=problem.name,
        instance_file_name = f.split('\\')[-1].split('.')[0],
        dataset_name='itc2019',
        date=date,
        results_data=res,
        experiment_data={
            "solver": 'depth_first_search',
            "max_operations": 10000
        }
    )
    
    pickle.dump(experiment_details, open(os.path.join(sub_folder, 'experiment_details.pkl'), 'wb'))
    
    if res['success']:
        np.save(os.path.join(sub_folder, 'solution gene.npy'), search.get_result_as_gene())
    
    plot_row_over_operation(os.path.join(sub_folder, 'row_over_operation_graph.png'), res['operation_history'], 'Depth first search max 10k operations for itc2019 - ' + experiment_details.instance_name)
    
    plot_row_over_time(os.path.join(sub_folder, 'row_over_time_graph.png'),res['operation_history'], 'Depth first search max 10k operations for itc2019 - ' + experiment_details.instance_name)
    

proceeded from 0 to 1
proceeded from 1 to 2
proceeded from 2 to 3
proceeded from 3 to 4
proceeded from 4 to 5
proceeded from 5 to 6
proceeded from 6 to 7
proceeded from 7 to 8
proceeded from 8 to 9
proceeded from 9 to 10
proceeded from 10 to 11
proceeded from 11 to 12
proceeded from 12 to 13
proceeded from 13 to 14
proceeded from 14 to 15
proceeded from 15 to 16
proceeded from 16 to 17
proceeded from 17 to 18
proceeded from 18 to 19
proceeded from 19 to 20
proceeded from 20 to 21
proceeded from 21 to 22
proceeded from 22 to 23
proceeded from 23 to 24
proceeded from 24 to 25
proceeded from 25 to 26
proceeded from 26 to 27
proceeded from 27 to 28
proceeded from 28 to 29
proceeded from 29 to 30
proceeded from 30 to 31
proceeded from 31 to 32
proceeded from 32 to 33
proceeded from 33 to 34
proceeded from 34 to 35
proceeded from 35 to 36
proceeded from 36 to 37
proceeded from 37 to 38
proceeded from 38 to 39
proceeded from 39 to 40
proceeded from 40 to 41
proceeded from 41 to 42
proceeded f

KeyboardInterrupt: 

NameError: name 'analysis_df' is not defined